# Global Mesh

This Notebook presents the workflow for developing global meshes. 


- Requires:

    - The data folder. See [README](README.md) in this folder.

In [ ]:
#to use the full width of the browser window
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Not neccessarily needed. Just to check version. Must be >= 0.6
import pyposeidon
pyposeidon.__version__

In [ ]:
import pyposeidon.mesh as pmesh

### Graphics

In [ ]:
import pyposeidon.utils.pplot

In [ ]:
import pyposeidon.utils.mplot

In [ ]:
#import pyposeidon.utils.hplot

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

In [ ]:
#optional
#pyposeidon.utils.pplot.__init__(dark_background=True) # set plt style for pplot graphics below

In [ ]:
#optional
#plt.style.use('dark_background')

In [ ]:
%matplotlib widget

In [ ]:
#Create a folder to save the output
import os
if not os.path.exists('test'):
            os.makedirs('test')

- In order to have within the Notebook the executable binaries installed in the conda env of the Kernel we add the corresponding path to the env `PATH`. This is not needed when we are running from python.

In [ ]:
import numpy as np
cpath = np.__path__[0].split('/lib/')[0] # get the current kernel path

os.environ['PATH'] += os.pathsep + cpath + '/bin' # add to PATH

## Setup

The minimum information required is the corresponding coastlines. They can be in a form that `geopandas` can read or in a `GeoDataFrame`.

In [ ]:
import geopandas as gp

In [ ]:
w = gp.read_file('./data/ocean/ocean.shp')

In [ ]:
w.plot()

We define "areametry" as "global" and the engine for grid generation ('jigsaw' or 'gmsh').

*Note*: There is no default and the user needs to specify one.

In [ ]:
mesh = pmesh.set(type='tri2d', geometry='global', coastlines = w, mesh_generator = 'gmsh', rpath='./test/global/' 
               ,use_bindings=True, #set this to False for gmsh binary
              )

In [ ]:
mesh.Dataset

**TIP** : You can change the grid generator above with  grid_generator = 'gmsh'.

## Visualize

- with mayavi/pyvista in 3D

In [ ]:
mesh.Dataset.mplot.mesh(dim='3D')

## Control mesh size

Better control on the mesh size can be obtained by providing a background control function usually in the form of a scaled DEM. One way to achieve this is to give as argument a dem file, like : 

In [ ]:
mesh = pmesh.set(type='tri2d', geometry='global', coastlines = w, mesh_generator = 'gmsh', 
                rpath = './test/global/',
                dem_source='./data/dem.nc', 
                resolution_min=0.001, resolution_max=.03,
                use_bindings=False, #set this to True for gmsh python bindings
               DistMin=.0, DistMax=.1, SizeMin=.005, SizeMax=.02, MeshSizeMin=.001,MeshSizeMax=0.05, # GMSH additional settings
              )

In [ ]:
mesh.boundary.contours.plot()

In [ ]:
mesh.bgmesh.pplot.mesh()

In [ ]:
mesh.bgmesh.pplot.contourf(var='h')

In [ ]:
mesh.Dataset.mplot.mesh(dim='3D')

In [ ]:
mesh.Dataset

**TIP** : Change the grid generator above with  grid_generator = 'jigsaw' to see how `jigsaw` would do it.

## Output

In [ ]:
mesh.to_file('./test/global/hgrid.gr3')

## Read a given grid

In [ ]:
m = pmesh.set(type='tri2d',mesh_file='./test/global/hgrid.gr3')

### Mesh Validation

If you model is SCHISM, the mesh can be validated against it as 

In [ ]:
m.validate(rpath='./test/global/val/')

### Grid Verification

The grid is created with a specific boundary defined, usually, as a coastline. In order to check whether that coastline is crossed by the generated grid we can use :

In [ ]:
mesh.verify(coastlines=w)

*Note* : This is still under development and there is no distiction between 'intersection' and 'touch' for the grid. That is, the nodes could very well be **on** the boundary as they should. 